Устанавливаем все необходимые библиотеки.

In [ ]:
from google.colab import files
from IPython.display import clear_output

In [ ]:
!pip install pycodestyle pycodestyle_magic
!pip install flake8
clear_output()

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Загружаем данные.

In [ ]:
whole_table_with_lemm = drive.CreateFile(
    {'id': '1rKrNCmgWPReJujCoZCDlhAb66yiUZ_by'})
whole_table_with_lemm.GetContentFile('whole_table_with_lemm.csv')

Импортируем все необходимые библиотеки.

In [ ]:
import re
import nltk
import joblib

import pandas as pd
import numpy as np

from pprint import pprint
from ast import literal_eval
from datetime import datetime

from tqdm.notebook import tqdm

from nltk.corpus import stopwords

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

Устанавливаем максимальную ширину столбца в датафрейме 100, чтобы лучше видеть данные.

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('russian'))

In [ ]:
# добавляем стоп-слова, обнаруженные в выделенных в прошлый раз топиках

new_stop_words = {
    'ваш', 'весь', 'который', 'наш', 'очень', 'самый', 'свой', 'твой', 'это'
    }

stop_words |= new_stop_words

In [ ]:
len(stop_words)

160

# Цель исследования
Тематическое моделирование корпуса текстов: связь тем дневниковых записей с периодом времени написания.
# Данные
**Meet your dataset: дневниковые записи** [(источник)](https://t.me/c/1358367494/782)<br>
<br>
**Откуда у нас дневники?**<br>
Наши данные - это причёсанный дамп [сайта "Прожито"](https://prozhito.org) от апреля 2019 года. Таблицы содержит несколько сотен тысяч записей за большой отрезок времени (от XVIII до XXI века, преимущественно — XX век), так что вам будет, где развернуться ;).<br>
<br>
**Как устроен датасет?**<br>
У нас есть две таблицы, `whole_table.csv` и `whole_table_with_lemm.csv`. Вторая отличается наличием колонки c лемматизированными (т.е. с приведенными в начальную форму словами) mystem записями.<br>
В обеих таблицах есть колонки:
*   `notes` - содержит сами дневниковые записи
*   `dates` - дата записи в формате год/месяц/день
*   `id`  -  айдишник автора (не записи!)
*   `author` - имя автора записи
<br>

Практически все поля заполнены, у некоторых отсутствует дата (так как таблица отсортирована по датам, они в самом начале).<br>
*От авторов исследования: даты отформатированы как YYYY/(M)M/(D)D, отсутствовать могут день, месяц или год, отсутствующие элементы даты заменены нулями.*
# Препроцессинг

In [ ]:
# напишем отдельную функцию парсинга дат для записей раньше 1905 или позже 2001 года
# или записей у которых отсутствует год, месяц или число
# исходя из целей исследования, такие записи неинформативны и их нужно удалять

def date_parser(date):
    if date and re.match(r'^(19|20)\d{2}/[1-9]\d?/[1-9]\d?$', date):
        date_split = [int(elem) for elem in date.split('/')]
        if date_split[0] >= 1905 and date_split[0] <= 2001:
            return datetime(year=date_split[0],
                            month=date_split[1],
                            day=date_split[2])
    return np.nan

In [ ]:
df = pd.read_csv('whole_table_with_lemm.csv', sep='\t', index_col=0,
                 usecols=[0, 2, 3], parse_dates=['dates'],
                 date_parser=date_parser).rename(
                     columns={'notes\n': 'lemm', 'dates': 'date'})

In [ ]:
df.head()

,lemm,date
0,"воскресение. \nвчера приезжать слава навещать мать в бол-це, но слезать с поезд в толмачев, г...",NaT
1,"ночь мы достигать банка антон-дорн, в датский пролив (между исландия и гренландия). утро погода ...",NaT
2,суббота. москва. ехать в селище.\n,NaT
3,"18 август его королевский величество отправляться из вильна в орша. в тот же день, высылать пере...",NaT
4,"на следующий день, 19 август, прибывать в остров, где не быть никакой известие.\n",NaT


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384526 entries, 0 to 384525
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   lemm    384525 non-null  object        
 1   date    304138 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 8.8+ MB


In [ ]:
!rm whole_table_with_lemm.csv

Удаляем ненужные записи, делаем столбец `date` индексом.

In [ ]:
df = df.dropna().set_index('date')

In [ ]:
df.head()

,lemm
date,
1905-01-01,"в 7 час утро — 4°, туман с дым, тихо. сегодня воскресение, оставаться дома. \nрано утро грохо..."
1905-01-01,с ранний утро японец опять открывать огонь разом по весь позиция наш крепость. \nсамый ожесточе...
1905-01-01,день новый год проходить благополучно. японец на хоутхайский сопка опять выкидывать свой флаг и ...
1905-01-01,"вчера говорить, что поражать перемена, который происходить в витта, что он становиться консерват..."
1905-01-01,"воскресение. японский выя год \nобычный воскресный богослужение, потом молебен, на который и ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 304137 entries, 1905-01-01 to 2001-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lemm    304137 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB


## Токенизация

In [ ]:
regex = r'[!\"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~— \n\d\xa0«»–❉№°]+'

In [24]:
# странные токены, обнаруженные в выделенных в прошлый раз топиках

non_words = {
    'аа', 'ак', 'ап', 'аран', 'бол', 'го', 'игада', 'олхо', 'оп', 'отд', 'ошай', 'пог'
    }

In [25]:
# почему-то в лемматизированных данных попадаются символы в верхнем регистре

def tokenization(text):
    return [elem for elem in re.split(regex, text.lower())
            if len(elem) > 1 # удаляем пустые токены и токены из одной буквы
            and elem not in stop_words # удаляем стоп-слова
            and elem not in non_words # удаляем странные токены
            and elem not in ['id', 'com']] # и токены id и com из ссылок

In [26]:
df['lemm'] = df['lemm'].apply(tokenization)

In [27]:
df.head()

,lemm
date,
1905-01-01,"[час, утро, туман, дым, тихо, сегодня, воскресение, оставаться, дома, рано, утро, грохотать, пуш..."
1905-01-01,"[ранний, утро, японец, открывать, огонь, разом, позиция, крепость, ожесточенный, бомбардировка, ..."
1905-01-01,"[день, новый, год, проходить, благополучно, японец, хоутхайский, сопка, выкидывать, флаг, кричат..."
1905-01-01,"[вчера, говорить, поражать, перемена, происходить, витта, становиться, консерватор, победоносцев..."
1905-01-01,"[воскресение, японский, выя, год, обычный, воскресный, богослужение, молебен, выходить, обычный,..."


Сохраняем и загружаем на гуглдиск готовые к topic modelling данные.

In [28]:
df.to_csv('prozhito_lemmas.csv')

In [29]:
!zip prozhito_lemmas.zip prozhito_lemmas.csv

  adding: prozhito_lemmas.csv (deflated 78%)


In [30]:
!rm prozhito_lemmas.csv

In [31]:
files.download('prozhito_lemmas.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки готовой к topic modelling таблицы

# prozhito_lemmas = drive.CreateFile(
#    {'id': '1RnIWAaw-oKkJF6Qmf7tHuii1-3AccxHG'})
# prozhito_lemmas.GetContentFile('prozhito_lemmas.zip')

Функция для загруженного из гуглдиска датафрейма.

In [32]:
def data_to_list(filename):
    df = pd.read_csv(filename)
    df['lemm'] = df['lemm'].apply(literal_eval)
    return df

## Периодизация
Поскольку основная часть текстов корпуса приходится на советскую эпоху, решено было выделить следующие периоды:
*   *янв. 1905 – янв. 1907* (первая революция)
*   *февр. 1907 – май 1914* (период между революциями)
*   *июнь 1914 – октябрь 1918* (Первая мировая война и две главные революции в России)
*   *ноябрь 1918 – декабрь 1927* (гражданская война, продразверстка, НЭП)
*   *январь 1928 – август 1939* (коллективизация, Большой террор)
*   *сентябрь 1939 – май 1945* (Вторая мировая война)
*   *1946 – 1963* (промежуточный период истории, заканчивается «оттепелью»)
*   *1964 – май 1987* (застой)
*   *июнь 1987 – июнь 1990* (перестройка и развал СССР)
*   *1991 – 2001* (после СССР, «лихие 90-е»)

In [33]:
df = [
    df.loc[
        pd.Timestamp('1905-1-1 00:00:00'): pd.Timestamp('1907-1-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1907-2-1 00:00:00'): pd.Timestamp('1914-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1914-6-1 00:00:00'): pd.Timestamp('1918-10-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1918-11-1 00:00:00'): pd.Timestamp('1927-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1928-1-1 00:00:00'): pd.Timestamp('1939-8-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1939-9-1 00:00:00'): pd.Timestamp('1945-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1946-1-1 00:00:00'): pd.Timestamp('1963-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1964-1-1 00:00:00'): pd.Timestamp('1987-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1987-6-1 00:00:00'): pd.Timestamp('1990-6-30 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1991-1-1 00:00:00'): pd.Timestamp('2001-12-31 23:59:59')]
       ]

In [34]:
df[1].head()

,lemm
date,
1907-02-01,"[пятница, чудный, погода, солнце, ветер, пахнуть, весна, удивительно, приятно, завтрак, лена, хо..."
1907-02-01,"[вчера, ната, звонить, шлиппе, сказать, сергей, александрович, уезжать, петербург, становиться, ..."
1907-02-01,"[пятница, приходиться, позвать, шенгелидзе, упорный, кашель, людмилка, проходить, доктор, напуга..."
1907-02-01,"[четверг, писать, февраль, доклад, военный, министр, ездить, город, мочь, день, следующий, прово..."
1907-02-01,"[тетя, васино, рождение, тетя, мила, ходить, кноп, гостиный, видеть, невский, студент, чичериных..."


In [35]:
period = ['first revolution', 'between revolutions', 'WWI', 'civil war',
          'great purge', 'WWII', 'before ottepel', 'stagnation','perestroyka',
          '90s']

In [36]:
for i, time in enumerate(period):
    print(f'{time}:{" " * (20 - len(time))}\t{df[i].shape[0]} entries')

first revolution:    	5821 entries
between revolutions: 	18761 entries
WWI:                 	20362 entries
civil war:           	35981 entries
great purge:         	37175 entries
WWII:                	62686 entries
before ottepel:      	39742 entries
stagnation:          	58121 entries
perestroyka:         	6723 entries
90s:                 	14874 entries


Сохраняем и загружаем на гуглдиск разбитые на периоды данные.

In [37]:
for i, table in tqdm(enumerate(df)):
    table.to_csv(f'{i + 1}_{period[i].replace(" ", "_")}.csv')

In [39]:
!zip prozhito_periodization.zip *.csv

  adding: 10_90s.csv (deflated 78%)
  adding: 1_first_revolution.csv (deflated 79%)
  adding: 2_between_revolutions.csv (deflated 79%)
  adding: 3_WWI.csv (deflated 79%)
  adding: 4_civil_war.csv (deflated 78%)
  adding: 5_great_purge.csv (deflated 78%)
  adding: 6_WWII.csv (deflated 78%)
  adding: 7_before_ottepel.csv (deflated 79%)
  adding: 8_stagnation.csv (deflated 78%)
  adding: 9_perestroyka.csv (deflated 78%)


In [40]:
files.download('prozhito_periodization.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки архива таблиц для каждого периода

# prozhito_periodization = drive.CreateFile(
#    {'id': '1ghZXSjw92FIzpAIewSDZGkfJf8IpJ7u9'})
# prozhito_periodization.GetContentFile('prozhito_periodization.zip')

# LDA with Gensim

In [41]:
models = []

In [42]:
for table in df:
    data = list(table['lemm'])
    id2word = Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    model = LdaModel(corpus, num_topics=10, id2word=id2word, passes=15,
                     iterations=100, per_word_topics=True)
    models.append(model)
    clear_output()
    print(f'Моделей обучено: {len(models)}')

Моделей обучено: 10


Сохраняем и загружаем на гуглдиск обученные модели с наборами тем для каждого периода.

In [43]:
joblib.dump(models, 'prozhito_topic_modelling.pkl')

['prozhito_topic_modelling.pkl']

In [44]:
!zip prozhito_topic_modelling.zip prozhito_topic_modelling.pkl

  adding: prozhito_topic_modelling.pkl (deflated 51%)


In [45]:
files.download('prozhito_topic_modelling.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки обученных моделей

# prozhito_topic_modelling = drive.CreateFile(
#    {'id': '1jBG3E4fAcjeXjkBGLq5kLAVo9y7njJOr'})
# prozhito_topic_modelling.GetContentFile('prozhito_topic_modelling.pkl')

# Результаты
[Визуализация результатов](https://colab.research.google.com/drive/1rejmaQAdDrRG6KLD1cb4T382OfaVs_Rh?usp=sharing)<br>
[Результаты в виде таймлайна](https://www.sutori.com/story/zhivyie-dushi-xx-vieka--LVLEntAwko13P3cmo1Chy9D8)
## Первая революция
### (*янв. 1905 – янв. 1907*)
Темы дневниковых записей:

In [46]:
pprint(models[0].print_topics())

[(0,
  '0.009*"говорить" + 0.007*"день" + 0.006*"царь" + 0.006*"сказать" + '
  '0.006*"сегодня" + 0.005*"рабочий" + 0.005*"человек" + 0.004*"государь" + '
  '0.004*"город" + 0.004*"министр"'),
 (1,
  '0.008*"год" + 0.005*"сегодня" + 0.005*"урок" + 0.005*"слово" + '
  '0.005*"читать" + 0.004*"первый" + 0.004*"написать" + 0.004*"горький" + '
  '0.004*"день" + 0.004*"класс"'),
 (2,
  '0.014*"говорить" + 0.010*"мочь" + 0.007*"сказать" + 0.006*"человек" + '
  '0.006*"дело" + 0.006*"знать" + 0.005*"ничто" + 0.004*"хотеть" + '
  '0.004*"давать" + 0.004*"год"'),
 (3,
  '0.014*"письмо" + 0.010*"павлик" + 0.010*"вечер" + 0.009*"сомов" + '
  '0.009*"читать" + 0.009*"писать" + 0.008*"день" + 0.008*"сережа" + '
  '0.008*"ребенок" + 0.007*"утро"'),
 (4,
  '0.008*"военнопленный" + 0.007*"вечер" + 0.007*"церковь" + 0.007*"день" + '
  '0.006*"письмо" + 0.006*"сегодня" + 0.005*"русский" + 0.005*"пойти" + '
  '0.005*"приходить" + 0.005*"россия"'),
 (5,
  '0.024*"жизнь" + 0.020*"человек" + 0.013*"мочь" + 

## Период между революциями
### (*февр. 1907 – май 1914*)
Темы дневниковых записей:

In [47]:
pprint(models[1].print_topics())

[(0,
  '0.019*"львовна" + 0.006*"аполлон" + 0.005*"наука" + 0.004*"картина" + '
  '0.004*"италия" + 0.004*"музей" + 0.003*"педагогика" + 0.003*"венеция" + '
  '0.003*"флоренция" + 0.003*"рим"'),
 (1,
  '0.014*"день" + 0.009*"жена" + 0.008*"утро" + 0.008*"павловск" + '
  '0.007*"вечер" + 0.006*"приезжать" + 0.006*"це" + 0.006*"час" + '
  '0.005*"корпус" + 0.005*"вернуться"'),
 (2,
  '0.029*"що" + 0.011*"як" + 0.011*"він" + 0.008*"коли" + 0.007*"його" + '
  '0.006*"бо" + 0.006*"аля" + 0.005*"щоба" + 0.005*"ми" + 0.004*"від"'),
 (3,
  '0.017*"театр" + 0.007*"сегодня" + 0.007*"говорить" + 0.007*"пьеса" + '
  '0.006*"спектакль" + 0.005*"сказать" + 0.005*"опера" + 0.005*"сцена" + '
  '0.005*"артист" + 0.005*"мочь"'),
 (4,
  '0.006*"идти" + 0.004*"большой" + 0.004*"становиться" + 0.004*"стоять" + '
  '0.004*"вода" + 0.003*"дорога" + 0.003*"солнце" + 0.003*"глаз" + '
  '0.003*"место" + 0.003*"земля"'),
 (5,
  '0.014*"вечер" + 0.010*"приходить" + 0.010*"пойти" + 0.008*"сидеть" + '
  '0.008*"игр

## Первая мировая война и две главные революции в России
###(*июнь 1914 – октябрь 1918*)
Темы дневниковых записей:

In [48]:
pprint(models[2].print_topics())

[(0,
  '0.012*"человек" + 0.012*"мочь" + 0.012*"жизнь" + 0.006*"знать" + '
  '0.006*"жить" + 0.006*"время" + 0.006*"душа" + 0.005*"думать" + 0.005*"день" '
  '+ 0.005*"год"'),
 (1,
  '0.025*"бо" + 0.013*"de" + 0.012*"вже" + 0.012*"вонь" + 0.010*"яковлевич" + '
  '0.010*"буде" + 0.008*"la" + 0.006*"et" + 0.006*"миф" + 0.006*"le"'),
 (2,
  '0.032*"що" + 0.012*"як" + 0.012*"він" + 0.009*"коли" + 0.008*"це" + '
  '0.007*"його" + 0.006*"гетьман" + 0.006*"аля" + 0.005*"щоба" + 0.005*"мена"'),
 (3,
  '0.011*"сережа" + 0.010*"день" + 0.010*"приходить" + 0.010*"вечер" + '
  '0.008*"пойти" + 0.008*"сегодня" + 0.008*"час" + 0.008*"говорить" + '
  '0.007*"мама" + 0.007*"утро"'),
 (4,
  '0.009*"большевик" + 0.008*"говорить" + 0.007*"день" + 0.006*"москва" + '
  '0.006*"газета" + 0.006*"россия" + 0.005*"народ" + 0.005*"сегодня" + '
  '0.005*"совет" + 0.005*"русский"'),
 (5,
  '0.006*"искусство" + 0.006*"музей" + 0.005*"картина" + 0.005*"эрмитаж" + '
  '0.004*"луначарский" + 0.004*"вообще" + 0.004*"р

## Гражданская война, продразверстка, НЭП
### (*ноябрь 1918 – декабрь 1927*)
Темы дневниковых записей:

In [50]:
pprint(models[3].print_topics())

[(0,
  '0.011*"говорить" + 0.010*"сказать" + 0.009*"день" + 0.007*"мочь" + '
  '0.007*"знать" + 0.006*"сегодня" + 0.006*"пойти" + 0.006*"приходить" + '
  '0.006*"вечер" + 0.006*"хотеть"'),
 (1,
  '0.007*"солнце" + 0.006*"вода" + 0.005*"белый" + 0.005*"большой" + '
  '0.005*"небо" + 0.005*"дом" + 0.005*"дерево" + 0.005*"церковь" + '
  '0.005*"стоять" + 0.005*"окно"'),
 (2,
  '0.015*"человек" + 0.012*"жизнь" + 0.010*"мочь" + 0.006*"становиться" + '
  '0.005*"время" + 0.005*"любовь" + 0.004*"душа" + 0.004*"знать" + '
  '0.004*"жить" + 0.004*"сила"'),
 (3,
  '0.013*"читать" + 0.010*"работа" + 0.008*"книга" + 0.006*"прочитывать" + '
  '0.005*"статья" + 0.005*"вечер" + 0.005*"написать" + 0.005*"театр" + '
  '0.005*"день" + 0.004*"доклад"'),
 (4,
  '0.025*"рб" + 0.013*"de" + 0.008*"инута" + 0.007*"la" + 0.006*"ия" + '
  '0.004*"et" + 0.004*"science" + 0.004*"le" + 0.003*"les" + 0.003*"des"'),
 (5,
  '0.008*"год" + 0.006*"говорить" + 0.006*"москва" + 0.005*"день" + '
  '0.005*"человек" + 0.004

## Коллективизация, Большой террор
### (*январь 1928 – август 1939*)
Темы дневниковых записей:

In [51]:
pprint(models[4].print_topics())

[(0,
  '0.006*"искусство" + 0.005*"картина" + 0.005*"фильм" + 0.004*"форма" + '
  '0.004*"новый" + 0.004*"образ" + 0.003*"большой" + 0.003*"тема" + '
  '0.003*"часть" + 0.003*"материал"'),
 (1,
  '0.024*"що" + 0.010*"як" + 0.007*"ми" + 0.006*"аля" + 0.006*"це" + '
  '0.006*"його" + 0.005*"він" + 0.005*"мена" + 0.005*"вже" + 0.005*"щоба"'),
 (2,
  '0.011*"хозяйка" + 0.010*"марья" + 0.010*"консультация" + 0.009*"колхоз" + '
  '0.008*"дома" + 0.008*"кирюша" + 0.007*"домашность" + 0.006*"письмо" + '
  '0.006*"свист" + 0.005*"катя"'),
 (3,
  '0.010*"километр" + 0.006*"пихта" + 0.005*"маршрут" + 0.004*"петрович" + '
  '0.004*"to" + 0.004*"лаборатория" + 0.004*"научный" + 0.004*"работа" + '
  '0.004*"гидрологический" + 0.004*"ический"'),
 (4,
  '0.005*"вопрос" + 0.005*"дело" + 0.005*"советский" + 0.005*"война" + '
  '0.004*"год" + 0.004*"мочь" + 0.004*"работа" + 0.004*"ссср" + '
  '0.004*"смоленское" + 0.003*"москва"'),
 (5,
  '0.012*"человек" + 0.011*"мочь" + 0.009*"жизнь" + 0.008*"говорить"

## Вторая мировая война
### (*сентябрь 1939 – май 1945*)
Темы дневниковых записей:

In [52]:
pprint(models[5].print_topics())

[(0,
  '0.011*"война" + 0.010*"германия" + 0.009*"мао" + 0.008*"берлин" + '
  '0.007*"советский" + 0.007*"год" + 0.007*"кпк" + 0.006*"цзэ" + '
  '0.006*"союзник" + 0.006*"дун"'),
 (1,
  '0.011*"работа" + 0.007*"вечер" + 0.006*"говорить" + 0.005*"дело" + '
  '0.005*"работать" + 0.005*"театр" + 0.005*"концерт" + 0.004*"давать" + '
  '0.004*"доклад" + 0.004*"первый"'),
 (2,
  '0.008*"сьогодні" + 0.007*"він" + 0.005*"дружина" + 0.005*"the" + '
  '0.005*"аля" + 0.004*"було" + 0.004*"кан" + 0.004*"мені" + 0.004*"від" + '
  '0.004*"був"'),
 (3,
  '0.012*"немец" + 0.010*"день" + 0.009*"фронт" + 0.009*"бой" + 0.008*"армия" '
  '+ 0.007*"командир" + 0.007*"полк" + 0.006*"войско" + 0.006*"противник" + '
  '0.006*"дивизия"'),
 (4,
  '0.010*"день" + 0.008*"говорить" + 0.008*"письмо" + 0.008*"мочь" + '
  '0.007*"знать" + 0.007*"сегодня" + 0.006*"год" + 0.006*"приходить" + '
  '0.006*"сказать" + 0.005*"ничто"'),
 (5,
  '0.007*"рука" + 0.007*"немец" + 0.007*"машина" + 0.006*"человек" + '
  '0.006*"ска

## Промежуточный период истории, заканчивается «оттепелью» 
### (*1946 – 1963*)
Темы дневниковых записей:

In [53]:
pprint(models[6].print_topics())

[(0,
  '0.007*"вода" + 0.006*"идти" + 0.005*"место" + 0.005*"дорога" + '
  '0.004*"большой" + 0.004*"берег" + 0.004*"лес" + 0.004*"становиться" + '
  '0.004*"земля" + 0.003*"машина"'),
 (1,
  '0.012*"бригада" + 0.007*"що" + 0.007*"колхоз" + 0.005*"корова" + '
  '0.005*"человек" + 0.005*"це" + 0.005*"кг" + 0.004*"колхозник" + '
  '0.004*"молоко" + 0.004*"він"'),
 (2,
  '0.015*"человек" + 0.010*"мочь" + 0.008*"жизнь" + 0.006*"время" + '
  '0.005*"год" + 0.005*"дело" + 0.005*"знать" + 0.004*"говорить" + '
  '0.004*"слово" + 0.004*"мир"'),
 (3,
  '0.017*"день" + 0.013*"час" + 0.013*"вечер" + 0.011*"утро" + 0.010*"пойти" + '
  '0.009*"сегодня" + 0.008*"приходить" + 0.006*"ходить" + 0.005*"приезжать" + '
  '0.005*"поехать"'),
 (4,
  '0.007*"вечер" + 0.007*"играть" + 0.006*"театр" + 0.006*"говорить" + '
  '0.006*"концерт" + 0.005*"музыка" + 0.005*"хороший" + 0.005*"школа" + '
  '0.005*"твардовский" + 0.004*"первый"'),
 (5,
  '0.016*"лидочка" + 0.011*"оч" + 0.008*"одинцов" + 0.008*"теркин" + '

## Застой
### (*1964 – май 1987*)
Темы дневниковых записей:

In [54]:
pprint(models[7].print_topics())

[(0,
  '0.011*"день" + 0.007*"час" + 0.007*"утро" + 0.006*"город" + 0.006*"идти" + '
  '0.006*"вечер" + 0.005*"дом" + 0.004*"дорога" + 0.004*"ночь" + 0.004*"вода"'),
 (1,
  '0.014*"работа" + 0.011*"вопрос" + 0.008*"ссср" + 0.007*"план" + 0.007*"цк" '
  '+ 0.007*"строительство" + 0.006*"министр" + 0.005*"совет" + '
  '0.005*"развитие" + 0.005*"страна"'),
 (2,
  '0.018*"вечер" + 0.012*"ходить" + 0.011*"спать" + 0.011*"работа" + '
  '0.009*"пьяный" + 0.009*"утро" + 0.009*"приезжать" + 0.008*"вставать" + '
  '0.008*"пить" + 0.008*"день"'),
 (3,
  '0.012*"эфрос" + 0.007*"колонна" + 0.006*"рука" + 0.005*"рота" + '
  '0.004*"глаз" + 0.003*"стол" + 0.003*"кричать" + 0.003*"обстреливать" + '
  '0.003*"стрельба" + 0.003*"голова"'),
 (4,
  '0.012*"стих" + 0.010*"фильм" + 0.009*"читать" + 0.009*"театр" + '
  '0.008*"спектакль" + 0.007*"играть" + 0.007*"поэт" + 0.007*"хороший" + '
  '0.006*"вечер" + 0.006*"слушать"'),
 (5,
  '0.012*"сказать" + 0.010*"день" + 0.010*"говорить" + 0.009*"год" + '
  '0.

## Перестройка и развал СССР
### (*июнь 1987 – июнь 1990*)
Темы дневниковых записей:

In [55]:
pprint(models[8].print_topics())

[(0,
  '0.009*"человек" + 0.008*"мочь" + 0.007*"говорить" + 0.006*"год" + '
  '0.006*"дело" + 0.005*"народ" + 0.005*"сказать" + 0.005*"жизнь" + '
  '0.004*"думать" + 0.004*"хотеть"'),
 (1,
  '0.007*"день" + 0.006*"человек" + 0.004*"машина" + 0.004*"время" + '
  '0.003*"утро" + 0.003*"знать" + 0.003*"сегодня" + 0.003*"идти" + 0.003*"час" '
  '+ 0.003*"ночь"'),
 (2,
  '0.012*"горбачев" + 0.009*"съезд" + 0.008*"совет" + 0.007*"вопрос" + '
  '0.006*"цк" + 0.005*"партия" + 0.005*"советский" + 0.004*"депутат" + '
  '0.004*"страна" + 0.004*"верховный"'),
 (3,
  '0.004*"клад" + 0.003*"прокофьев" + 0.003*"of" + 0.003*"ива" + 0.002*"гел" + '
  '0.002*"регина" + 0.002*"харджиев" + 0.002*"цирк" + 0.002*"бетховен" + '
  '0.002*"эстонский"'),
 (4,
  '0.009*"день" + 0.006*"час" + 0.006*"говорить" + 0.006*"сказать" + '
  '0.005*"приезжать" + 0.005*"вечер" + 0.004*"утро" + 0.004*"сегодня" + '
  '0.004*"приходить" + 0.004*"жить"'),
 (5,
  '0.006*"год" + 0.005*"летать" + 0.004*"экипаж" + 0.004*"полет" + 

## После СССР, «лихие 90-е»
### (*1991 – 2001*)
Темы дневниковых записей:

In [56]:
pprint(models[9].print_topics())

[(0,
  '0.010*"книга" + 0.009*"год" + 0.006*"дом" + 0.005*"город" + 0.005*"москва" '
  '+ 0.004*"музей" + 0.004*"русский" + 0.004*"новый" + 0.004*"день" + '
  '0.003*"деньги"'),
 (1,
  '0.010*"мочь" + 0.007*"сказать" + 0.007*"год" + 0.007*"говорить" + '
  '0.006*"время" + 0.006*"знать" + 0.005*"человек" + 0.005*"дело" + '
  '0.005*"понимать" + 0.004*"писать"'),
 (2,
  '0.018*"человек" + 0.015*"жизнь" + 0.010*"сказать" + 0.009*"мочь" + '
  '0.007*"душа" + 0.007*"бог" + 0.007*"говорить" + 0.007*"стих" + '
  '0.006*"любить" + 0.006*"жить"'),
 (3,
  '0.007*"день" + 0.006*"мама" + 0.006*"дом" + 0.004*"идти" + 0.004*"рука" + '
  '0.004*"становиться" + 0.004*"утро" + 0.004*"час" + 0.004*"вода" + '
  '0.003*"ночь"'),
 (4,
  '0.012*"сказать" + 0.010*"вечер" + 0.010*"читать" + 0.008*"звонить" + '
  '0.008*"день" + 0.007*"сегодня" + 0.007*"петь" + 0.006*"дома" + 0.006*"утро" '
  '+ 0.005*"позвонить"'),
 (5,
  '0.009*"полет" + 0.008*"экипаж" + 0.007*"летать" + 0.007*"скорость" + '
  '0.006*"давать